# Macroprudential Policy and the Housing Market - Replication Notebook
This notebook solves and simulates the Heterogenous Agent Housing Market (HAH) model and produces the output contained in my Master's thesis. 

**Dependencies:**

The code structure builds upon the framework developed by Jeppe Druedahl & Co. in the [NumEconCopenhagen Project](https://github.com/NumEconCopenhagen)

Packages required for running the notebooks are:
- [ConSav](https://pypi.org/project/ConSav/)
- [EconModel](https://pypi.org/project/EconModel/)
- [matplotlib](https://pypi.org/project/matplotlib/)
- [numpy](https://pypi.org/project/numpy/)
- [numba](https://pypi.org/project/numba/)
- [pandas](https://pypi.org/project/pandas/) ?
- [quantecon](https://pypi.org/project/quantecon/) ?

In [1]:
# imports
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import numba as nb
nb.set_num_threads(4)

from HAHModel import HAHModelClass 

## Baseline model

In [2]:
model_baseline = HAHModelClass(
    name='baseline',
    par={
        #'solmethod':'negm',
        #'cross_compute':True, # whether to cross-compute MPC if discrete choice is altered
        'do_print':True
        })

#model_baseline.precompile_numba() # solve with very coarse grids
#model_baseline.solve() 
#model_baseline.simulate()

### Testing utility functions, tax scheme and mortgage payment

In [3]:
import mt
import utility
from EconModel import jit

with jit(model_baseline) as model_jit:
    par = model_jit.par
    
    # mortgage schedule
    annuity,pr_rem,interest,pr_pmt = mt.mpmt(10**6,1,20,0,par)

    # income and property tax
    tax_h = mt.property_tax(1.0,2.0,par)
    ytilde = mt.income_tax(2.2,0.8,3.5,0,par)

    # utility
    u_stay = utility.func(1,2,0,0,5,par)
    marg_u_stay = utility.marg_func(2,par,par.T-1)
    u_move = utility.func(1,2,1,0,5,par)
    u_rent = utility.func(1,2,1,1,5,par)

# print output
print(f'the full annuity payment is {round(annuity,2)}')
print(f'the remaining principal is {round(pr_rem,2)}')
print(f'the interest payment is {round(interest,2)}')
print(f'the principal payment is {round(pr_pmt,2)}')
print('--------------------------------------------')
print(f'the property tax is {round(tax_h,2)}')
print(f'the after tax income is {round(ytilde,2)}')
print('--------------------------------------------')
print(f'utility for a stayer is {round(u_stay,2)}')
print(f'marginal stay utility is {round(marg_u_stay,2)}')
print(f'utility for a mover is {round(u_move,2)}')
print(f'utility for a renter is {round(u_rent,2)}')


the full annuity payment is 67215.71
the remaining principal is 962784.29
the interest payment is 30000.0
the principal payment is 37215.71
--------------------------------------------
the property tax is 0.02
the after tax income is 1.57
--------------------------------------------
utility for a stayer is -3.17
marginal stay utility is 0.18
utility for a mover is -3.51
utility for a renter is -3.55


### Testing transition rules

In [4]:
import trans
# cash on hand

# da periods
Tda_next = trans.Tda_plus_func(10)
print(Tda_next)

with jit(model_baseline) as model_jit:
    par = model_jit.par
    
    # mortgage balance
    d_plus = trans.d_plus_func(q=1,h=2,d=1.5,w=0.8,move=1,ref=0,t=5,Td=20,Tda=0,par=par)

print(f'next period mortgage balance is {d_plus}') 

# income states

9
next period mortgage balance is 1.6480000000000001


### Test last period solver

In [12]:
#own_shape = (par.T,par.Nm,par.Nh,par.Nd,par.Td_bar,par.Tda_bar,par.Nw)
#rent_shape = (par.T,par.Nm,par.Nw)
#post_shape = (par.T-1,par.Na,par.Nh,par.Nd,par.Td_bar,par.Tda_bar,par.Nw)
import trans
from HHproblems import last_period_v_bar_q

t = par.T-1

with jit(model_baseline) as model: 
    sol = model.sol
    par = model.par
    last_period_v_bar_q(t,sol,par)

    #assert np.all((sol.c_stay[t] >= 0) & (np.isnan(sol.c_stay[t]) == False))
    #assert np.all((sol.inv_v_bar[t] >= 0) & (np.isnan(sol.inv_v_bar[t]) == False))



Traceback (most recent call last):
  File "/var/folders/23/m5wgdyb92cs9fl4fls0nt3xw0000gn/T/ipykernel_9799/3358002345.py", line 15, in <cell line: 9>
    assert np.all((sol.inv_v_bar[t] >= 0) & (np.isnan(sol.inv_v_bar[t]) == False))
AssertionError


AssertionError: 

In [13]:
#assert np.all((sol.inv_v_stay[t] >= 0) & (np.isnan(sol.inv_v_stay[t]) == False))
#model_baseline.sol.inv_v_stay[par.T-1,:,0,0,0,0,0]

np.count_nonzero(np.isnan(sol.inv_v_bar[t]))
#sol.inv_v_bar[t,1,2,0,0,0,6]
#sol.inv_v_bar[t].shape
#-1/par.beta*utility.bequest_func(trans.ab_plus_func(1.5,par.grid_h[3],par),par,t)

#par.grid_a.round(1)

9490600

In [90]:
1/utility.marg_func(model_baseline.sol.c_stay[par.T-1,55,0,0,0,0,6],par,par.T-1)

#model_baseline.sol.c_stay[par.T-1,55,0,0,0,0,6]

0.0

In [46]:
model_baseline.par.w_grid 
model_baseline.par.w_trans.round(2)

array([[0.89, 0.11, 0.01, 0.  , 0.  , 0.  , 0.  ],
       [0.02, 0.89, 0.09, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.04, 0.89, 0.07, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.05, 0.89, 0.05, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.07, 0.89, 0.04, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.09, 0.89, 0.02],
       [0.  , 0.  , 0.  , 0.  , 0.01, 0.11, 0.89]])